### Yield From - Throwing Exceptions

We have seen that `yield from` allows us to establish a 2-way communication channel with a subgenerator and we could use `next`, and `send` to send a "request" to a delegated subgenerator via the delegator generator.

In fact, we can also send exceptions by throwing an exception into the delegator, just like a `send`.

In [ ]:
class CloseCoroutine(Exception):
    pass

def echo():
    try:
        while True:
            received = yield
            print(received)
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called')

In [ ]:
e = echo()
next(e)

In [ ]:
e.throw(CloseCoroutine, 'just closing')

In [ ]:
e = echo()
next(e)
e.close()

As we can see the difference between `throw` and `close` is that although `close` causes an exception to be raised in the generator, Python essentially silences it.

It works the same way when we delegate to the coroutine in a delegator:

In [ ]:
def delegator():
    result = yield from echo()
    yield 'subgen closed and returned:', result
    print('delegator closing...')

In [ ]:
d = delegator()
next(d)
d.send('hello')

In [ ]:
d.throw(CloseCoroutine)

Now what happens if the `throw` in the subgenerator does not close subgenerator but instead silences the exception and yields a value instead?

In [ ]:
class CloseCoroutine(Exception):
    pass

class IgnoreMe(Exception):
    pass

def echo():
    try:
        while True:
            try:
                received = yield
                print(received)
            except IgnoreMe:
                yield "I'm ignoring you..."
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called')

In [ ]:
d = delegator()
next(d)

In [ ]:
d.send('python')

In [ ]:
result = d.throw(IgnoreMe, 1000)

In [ ]:
result

In [ ]:
d.send('rocks!')

Why did we not get a yielded value back?

That's because the subgenerator was paused at the yield that yielded "I'm, ignoring you".

If we want to coroutine to continue running normally after ignoring that exception we need to tweak it slightly:

Let's first make sure we close our previous delegator!

In [ ]:
d.close()

In [ ]:
def echo():
    try:
        output = None
        while True:
            try:
                received = yield output
                print(received)
            except IgnoreMe:
                output = "I'm ignoring you..."
            else:
                output = None
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called')

In [ ]:
d = delegator()
next(d)

In [ ]:
d.send('hello')

In [ ]:
d.throw(IgnoreMe)

In [ ]:
d.send('python')

In [ ]:
d.close()

What happens if we do not handle the error in the subgenerator and simply let the exception propagate up?
Who gets the exception, the delegator, or the caller?

In [ ]:
def echo():
    while True:
        received = yield
        print(received)

In [ ]:
def delegator():
    yield from echo()

In [ ]:
d = delegator()
next(d)

In [ ]:
d.throw(ValueError)

OK, so we, the caller see the exception. But did the delegator see it too? i.e. can we catch the exception in the delegator?

In [ ]:
def delegator():
    try:
        yield from echo()
    except ValueError:
        print('got the value error')

In [ ]:
d = delegator()
next(d)

In [ ]:
d.throw(ValueError)

As you can see, we were able to catch the exception in the delegator.
Of course, the way we wrote our code, the delegator still closed, and hence we now see a `StopIteration` exception.

#### Example

Suppose we have a coroutine that creates running averages, and we want to occasionally write the current data to a file:

In [ ]:
class WriteAverage(Exception):
    pass

def averager(out_file):
    total = 0
    count = 0
    average = None
    with open(out_file, 'w') as f:
        f.write('count,average\n')
        while True:
            try:
                received = yield average
                total += received
                count += 1
                average = total / count
            except WriteAverage:
                if average is not None:
                    print('saving average to file:', average)
                    f.write(f'{count},{average}\n')

In [ ]:
avg = averager('sample.csv')
next(avg)

In [ ]:
avg.send(1)
avg.send(2)

In [ ]:
avg.throw(WriteAverage)

In [ ]:
avg.send(3)

In [ ]:
avg.send(2)

In [ ]:
avg.throw(WriteAverage)

In [ ]:
avg.close()

Now we can read the data back and make sure it worked as expected:

In [ ]:
with open('sample.csv') as f:
    for row in f:
        print(row.strip())

Of course we can use a delegator as well.
Maybe the delegator is charged with figuring out the output file name.
Here we'll just hardcode it inside the delegator:

In [ ]:
def delegator():
    yield from averager('sample.csv')

In [ ]:
d = delegator()
next(d)

In [ ]:
d.send(1)

In [ ]:
d.send(2)

In [ ]:
d.send(3)

In [ ]:
d.send(4)

In [ ]:
d.throw(WriteAverage)

In [ ]:
d.send(5)

In [ ]:
d.throw(WriteAverage)

In [ ]:
d.close()

In [ ]:
with open('sample.csv') as f:
    for row in f:
        print(row.strip())